### This notebook pre-processes the .csv files obtained from simulation to .pt dataset file.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install -q git+https://github.com/snap-stanford/deepsnap.git

2.0.1+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 28.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done


In [3]:
import random
import pandas as pd
import torch_scatter
import torch.nn as nn
from torch.nn import Linear, Sequential, LayerNorm, ReLU
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.data import DataLoader

import numpy as np
import time
import torch.optim as optim
from tqdm import trange
import pandas as pd
import copy
import matplotlib.pyplot as plt
import os

In [4]:
import functools
import json
from torch_geometric.data import Data
import enum
import itertools
import warnings
warnings.filterwarnings('ignore')

In [5]:
def triangles_to_edges(faces):
  # Collect edges from triangles
  edges = torch.cat([faces[:, 0:2],
                      faces[:, 1:3],
                      torch.stack([faces[:, 2], faces[:, 0]], dim=1)], dim=0)
  # Those edges are sometimes duplicated (within the mesh) and sometimes
  # single (at the mesh boundary).
  # Sort & pack edges as single torch.int64
  receivers = torch.min(edges, dim=1)[0]
  senders = torch.max(edges, dim=1)[0]
  packed_edges = torch.stack([senders, receivers], dim=1).to(torch.int64)
  # Remove duplicates and unpack
  unique_edges = torch.unique(packed_edges, dim=0).to(torch.int32)
  senders, receivers = unique_edges[:, 0], unique_edges[:, 1]
  # Create two-way connectivity
  return torch.cat([senders, receivers], dim=0), torch.cat([receivers, senders], dim=0)

from pandas._libs import internals

# This class defines the node type you want to classify
# For example, internal nodes/surface nodes/nodes under loading/fixed nodes, you can adjust this function based on your problem
class NodeType(enum.IntEnum):

    internal = 0
    upper_lower = 1
    SIZE = 2

In [6]:
data_file="/content/drive/MyDrive/hyundai_collaboration/Code_summary/data_preprocessing/sample_data_folder"

file_list=os.listdir(data_file)
len(file_list)

2

In [12]:
def create_dataset(data_file):
    data_list=[]
    file_name_list=[]
    file_list=os.listdir(data_file)
    j=1
    for item in file_list:

        design_name=item.rpartition('_')[0]
        print(j)
        if design_name not in file_name_list and design_name != ".DS" :

            info=design_name.split("_")
            #extra_features=torch.tensor([float(info[-5])*1e-9, float(info[-3])*1e-3, float(info[-1])*np.pi/180])
            #extra_features=torch.tensor([float(info[-5])*1e-11, float(info[-3])*1e-5])

            file_name_list.append(design_name)

            design_node=data_file+'/'+design_name+'_nodes.csv'
            design_element=data_file+'/'+design_name+'_elements.csv'

            df = pd.read_csv(design_node)
            df_node_indices_four=pd.read_csv(design_element)

            # four node 3D element to four 3nodel faces

            df_node_indices_three = pd.DataFrame( columns = ['elem1','elem2','elem3'])
            df_node_indices_three2 = pd.DataFrame( columns = ['elem1','elem2','elem3'])


            for i in range(df_node_indices_four.shape[0]):
                four_nodes=df_node_indices_four[i:i+1].values.tolist()[0][-4:]

                three_nodes=np.array(list(itertools.combinations(four_nodes,3)))
                cell_df=pd.DataFrame(three_nodes, columns = ['elem1','elem2','elem3'])
                df_node_indices_three = df_node_indices_three.append(cell_df)

                three_nodes2=np.array(list(itertools.permutations(four_nodes,3)))
                cell_df2=pd.DataFrame(three_nodes2, columns = ['elem1','elem2','elem3'])
                df_node_indices_three2 = df_node_indices_three2.append(cell_df2)



            cells_index2=np.vstack((df_node_indices_three2["elem1"].to_numpy(),df_node_indices_three2["elem2"].to_numpy(),df_node_indices_three2["elem3"].to_numpy())).T.astype('int32')
            cells2=torch.tensor(cells_index2)


            # cells (element indices)
            cells_index=np.vstack((df_node_indices_three["elem1"].to_numpy(),df_node_indices_three["elem2"].to_numpy(),df_node_indices_three["elem3"].to_numpy())).T.astype('int32')
            cells=torch.tensor(cells_index)

            # node positions
            node_position=np.vstack((df["x"].to_numpy(),df["y"].to_numpy(),df["z"].to_numpy())).T.astype('float32')
            mesh_pos=torch.tensor(node_position)

            # disp, strain, stress information
            stress=torch.tensor(np.vstack((df["S11"].to_numpy(),df["S22"].to_numpy(),df["S33"].to_numpy(),df["S12"].to_numpy(),df["S13"].to_numpy(),df["S12.1"].to_numpy())).T.astype('float32'))
            stress_mises=torch.unsqueeze(torch.tensor(df["mises"].to_numpy().T.astype('float32')), 1)
            stress11=torch.unsqueeze(torch.tensor(df["S11"].to_numpy().T.astype('float32')), 1)
            stress22=torch.unsqueeze(torch.tensor(df["S22"].to_numpy().T.astype('float32')), 1)
            stress33=torch.unsqueeze(torch.tensor(df["S33"].to_numpy().T.astype('float32')), 1)
            stress12=torch.unsqueeze(torch.tensor(df["S12"].to_numpy().T.astype('float32')), 1)
            stress13=torch.unsqueeze(torch.tensor(df["S13"].to_numpy().T.astype('float32')), 1)
            stress23=torch.unsqueeze(torch.tensor(df["S12.1"].to_numpy().T.astype('float32')), 1)


            # get edge indices
            edges = triangles_to_edges(torch.tensor(cells_index))
            edge_index = torch.cat( (torch.tensor(edges[0].numpy()).unsqueeze(0) ,
                         torch.tensor(edges[1].numpy()).unsqueeze(0)), dim=0).type(torch.long)

            # get edge features
            u_i=torch.tensor(node_position)[edge_index[0]]
            u_j=torch.tensor(node_position)[edge_index[1]]
            u_ij=u_i-u_j
            u_ij_norm = torch.norm(u_ij,p=2,dim=1,keepdim=True)
            edge_attr = torch.cat((u_ij,u_ij_norm),dim=-1).type(torch.float)

            # get input x

            node_type_info=df["nodetype"].to_numpy().T.astype('int32')
            print(node_type_info)
            print(node_type_info.shape)
            #node_type = torch.tensor(np.array(nn.functional.one_hot(torch.tensor(node_type_info), num_classes=NodeType.SIZE))).squeeze(1)
            node_type = nn.functional.one_hot(torch.tensor(node_type_info).to(torch.long), num_classes=NodeType.SIZE)
            x_without_features = torch.cat((mesh_pos,node_type),dim=-1).type(torch.float)

            #append extra_features to x
            #extra_features_stack=extra_features.repeat(x_without_features.shape[0],1)
            #x_=torch.cat((mesh_pos,extra_features_stack),dim=-1).type(torch.float)
            #x=torch.cat((x_,node_type),dim=-1).type(torch.float)
            #x = x_without_features

            # get data ist
            data_list.append(Data(x=x_without_features, x_without_features=x_without_features ,
                                  edge_index=edge_index, edge_attr=edge_attr, stress=stress,
                                  cells=cells, cells2=cells2,mesh_pos=mesh_pos, stress11=stress11,
                                 stress22=stress22, stress33=stress33, stress12=stress12,
                                 stress13=stress13, stress23=stress23,stress_mises=stress_mises, name=design_name ))
        else:
            pass
        j=j+1
    print("Done collecting data!")


    return torch.save(data_list, os.path.join("/content/drive/MyDrive/hyundai_collaboration/Code_summary/data_preprocessing/" + '3D_data_sample_pytorch.pt'))


create_dataset(data_file)

1
[1 1 1 ... 0 0 0]
(3361,)
2
Done collecting data!


In [13]:
dataset = torch.load("/content/drive/MyDrive/hyundai_collaboration/Code_summary/data_preprocessing/3D_data_sample_pytorch.pt")[0:1]

print(dataset)

[Data(x=[3361, 5], edge_index=[2, 37590], edge_attr=[37590, 4], x_without_features=[3361, 5], stress=[3361, 6], cells=[52720, 3], cells2=[316320, 3], mesh_pos=[3361, 3], stress11=[3361, 1], stress22=[3361, 1], stress33=[3361, 1], stress12=[3361, 1], stress13=[3361, 1], stress23=[3361, 1], stress_mises=[3361, 1], name='ro_0.226_ri_0.180_w_0.135_sw_0.031_n_4_E_10000000000.0_l_10000.0_rot_0.0')]


In [14]:

dataset = torch.load("/content/drive/MyDrive/hyundai_collaboration/Code_summary/data_preprocessing/3D_data_sample.pt")[0:1]

print(dataset)

[Data(x=[3361, 5], edge_index=[2, 37590], edge_attr=[37590, 4], x_without_features=[3361, 5], stress=[3361, 6], cells=[52720, 3], cells2=[316320, 3], mesh_pos=[3361, 3], stress11=[3361, 1], stress22=[3361, 1], stress33=[3361, 1], stress12=[3361, 1], stress13=[3361, 1], stress23=[3361, 1], stress_mises=[3361, 1], name='ro_0.226_ri_0.180_w_0.135_sw_0.031_n_4_E_10000000000.0_l_10000.0_rot_0.0')]


#### * a meshed wheel structure with n nodes has a nodal input size of n×5 (see "x") for the “Original Dataset” without augmented features such as loading force or rotation angle information (e.g. row_i:[x_i,y_i,z_i ]+[node type])

#### * An edge input has a size of m×4 (see "edge_attr") for a meshed wheel structure with m edges across all the dataset. The output size is always the same with a size of n×6 (see "stress") for stresses prediction.

#### *  Now, the .pt file will be our processed dataset to be used for traning and testing.


